# install important Packages

In [2]:
## remove teh comments to install teh required libraries
# %pip install opencv-python cvzone
# %pip install mediapipe --user

# Prep Work for Video save
 1. Download the ffmpeg build from this [Link](https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-win64-gpl.zip) (**Recomended for simplicity**),  
  or you can Go to the Repo and get the latest Build [Here](https://github.com/BtbN/FFmpeg-Builds/releases)
 2. Extract the `.Zip` folder and add the `bin` folder to your path environment variables
 3. after that close your currnet `shell` or `cmd`  and RUN `ffmpeg --version` to make sure it's corctly installed 
 4. install the python bindings for ffmpeg with 
```bash
    pip install ffmpeg-python
```

In [3]:
## command to install ffmpeg-python within your NoteBook
## remove teh comments to install teh required libraries
#%pip install ffmpeg-python

In [15]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot
import numpy as np
import ffmpeg

path = r"D:\New folder (5)\Video.mp4"
# *************************************** for real videocam
path = 0  # for real time feed
# ***************************************
cap = cv2.VideoCapture(path)
detector = FaceMeshDetector(maxFaces=1)
plotY = LivePlot(640, 360, [20, 50], invert=True)
idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
ratioList = []
blinkCounter = 0
counter = 0
color = (255, 0, 0)
blank = np.zeros((500, 500, 1), dtype="uint8")


# ******************************************************************
# Create a ffmpeg stream object for writing frames to the output file using the given format
OUTPUT_FILE_NAME = "output.mp4"  # Output file name
VideoCODEC = 'libx264'  # Output file codec
process = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='bgr24', s=f'{1280}x{360}')
    .output(str(OUTPUT_FILE_NAME), pix_fmt='yuv420p', vcodec=VideoCODEC)
    .overwrite_output()
    .run_async(pipe_stdin=True)
)
# *****************************************************************
while True:
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    success, img = cap.read()
    img, faces = detector.findFaceMesh(img, draw=True)
    if faces:
        face = faces[0]
        for id in idList:
            cv2.circle(img, face[id], 5, color, cv2.FILLED)
        leftUp = face[159]
        leftDown = face[23]
        leftLeft = face[130]
        leftRight = face[243]
        lenghtVer, _ = detector.findDistance(leftUp, leftDown)
        lenghtHor, _ = detector.findDistance(leftLeft, leftRight)
        cv2.line(img, leftUp, leftDown, (255, 200, 0), 3)
        cv2.line(img, leftLeft, leftRight, (0, 200, 0), 3)
        ratio = int((lenghtVer / lenghtHor) * 100)
        ratioList.append(ratio)
        if len(ratioList) > 3:
            ratioList.pop(0)
        ratioAvg = sum(ratioList) / len(ratioList)
        if 39 < ratioAvg < 43 and counter == 0:
            blinkCounter += 1
            color = (0, 200, 0)
            counter = 1
        if counter != 0:
            counter += 1
            if counter > 5:
                counter = 0
                color = (255, 0, 255)
        cvzone.putTextRect(img, f'Blink Count: {blinkCounter}', (50, 100),
                           colorR=color)
        if 5 < blinkCounter < 10:
            cvzone.putTextRect(img, 'real ', (50, 150), colorR=color)
        elif blinkCounter > 10:
            blinkCounter = 0
        imgPlot = plotY.update(ratioAvg, color)
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, imgPlot], 2, 1)
    else:
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, blank], 2, 1)
    # *********************************************************
    # Write each frame to output file using ffmpeg stream object
    process.stdin.write(
        imgStack
        .astype(np.uint8)
        .tobytes()
    )
    # *********************************************************
    cv2.imshow("Image", imgStack)
    if cv2.waitKey(30) == ord('q'):
        break
# ******************************************
process.stdin.close()
process.wait()
# ******************************************
cap.release()
cv2.destroyAllWindows()


## 